# Convert custom data using PyNWB

Previously you learned how to convert electrophysiology data from a common data type (Neuropixels) to the NWB format, using the [GUIDE](https://nwb-guide.readthedocs.io/en/latest).

In this tutorial we'll learn how to use [PyNWB](https://pynwb.readthedocs.io/en/stable/index.html) to add custom trials data to an existing NWB file. 

Let's start by installing the necessary packages.

In [ ]:
%%bash

pip install -U pynwb==2.7.0 dandi==0.62.1 nwbinspector==0.4.35 pyyaml pandas

# Download NWB file from DANDI archive (staging)

To download the dandiset containing the NWB files, we can use the [dandi cli](https://www.dandiarchive.org/handbook/12_download/#using-the-python-cli-client). The command below will create a folder in your current working directory containing all NWB files from the chosen dandiset. 

Remember to **change the dandiset id number**, choose the dandiset you created for the GUIDE tutorial.

In [ ]:
# Set your dandiset number here
dandiset_id = "213840"

# Run the bash command
bash_command = f'dandi download https://gui-staging.dandiarchive.org/dandiset/{dandiset_id}/draft'

!{bash_command}

## Open Downloaded file and explore it using PyNWB

Let's open the downloaded file with PyNWB and verify its contents. Remember to **change the path** to the file you just downloaded.

In [ ]:
import pynwb


# Change this to the path to the NWB file you just downloaded
file_path = "213840/sub-id0123/sub-id0123_ses-id987_ecephys.nwb"

# Open the file with PyNWB and verify its contents
read_io = pynwb.NWBHDF5IO(file_path, mode="r")
nwbfile = read_io.read()
nwbfile

In [ ]:
# You can access all data and information inside the nwbfile object
print("Electrical series recording information:")
print(nwbfile.acquisition['ElectricalSeriesAP'])

print("Subject information:")
print(nwbfile.subject)

In [ ]:
# Tabular information such as electrodes or sorted units can be explored as Pandas dataframes
nwbfile.electrodes.to_dataframe()

In [ ]:
# Let's check the Units table
nwbfile.units.to_dataframe()

When opening a file with PyNWB without a context manager (as we've just done in the code above) it is always important to **close the file**.

In [ ]:
# Remember to close the file if you are exploring it outside of the context manager mode
read_io.close()

# Add data to the original NWB file

The NWB file you created for the GUIDE tutorial contains raw electrophysiology traces and processed spiking data. With PyNWB you can include more experimental data to the file.

Next, you will add trials information to this file. Read more about adding and removing data from NWB files [here](https://pynwb.readthedocs.io/en/stable/tutorials/general/add_remove_containers.html). In this session we will:
- Open the NWB file in append mode.
- Add trials data. Read more about time intervals in NWB [here](https://pynwb.readthedocs.io/en/stable/tutorials/general/plot_file.html#time-intervals).
- Write the modified NWB file.
- Open the file and explore the new content in it
- Close the file

In [ ]:
# Open the file in `r+` mode, that allows us to add new data to it
with pynwb.NWBHDF5IO(file_path, mode="a") as io:
    nwbfile = io.read()

    # Create trials data and add them to the NWBFile object
    nwbfile.add_trial_column(
        name="correct",
        description="whether the trial was correct",
    )
    nwbfile.add_trial(start_time=0.0, stop_time=5.0, correct=True)
    nwbfile.add_trial(start_time=5.0, stop_time=10.0, correct=False)
    nwbfile.add_trial(start_time=10.0, stop_time=15.0, correct=True)

    # write the modifications to disk
    io.write(nwbfile)

In [ ]:
import pynwb
# Read the NWB file and confirm the modifications
read_io = pynwb.NWBHDF5IO(file_path, mode="r")
nwbfile = read_io.read()
nwbfile

In [ ]:
# Let's check the newly created Trials table
nwbfile.trials.to_dataframe()

In [ ]:
# Remember to close the file if you are exploring it outside of the context manager mode
read_io.close()

# Create a new NWB file based on the original NWB file
Many times it can be useful to modify the original NWB file, e.g. to include new processed data or to exclude raw data.

Next, you will create a new NWB file based on the original file, add new processed data and remove the raw traces from it. Read more about adding and removing data from NWB files [here](https://pynwb.readthedocs.io/en/stable/tutorials/general/add_remove_containers.html). In this session we will:
- Open the NWB file in read mode.
- Remove the raw electrophysiology traces.
- Add mock processed behavioral data. Read more about behavioral data in NWB [here](https://pynwb.readthedocs.io/en/stable/tutorials/domain/plot_behavior.html).
- Write the modified NWB object as a new NWB file.
- Open the new file and explore the content in it
- Close the file

In [ ]:
from utils import get_file_size_in_mb
import numpy as np


# Open the file in `r` mode, generate new unique id
with pynwb.NWBHDF5IO(file_path, mode="r") as read_io:
    nwbfile = read_io.read()
    nwbfile.generate_new_id()

    # Remove the raw traces from the NWBFile object
    nwbfile.acquisition.pop("ElectricalSeriesAP")

    # Create a Behavior Processing Module
    behavior_module = nwbfile.create_processing_module(
        name="behavior",
        description="Processed behavioral data"
    )
    
    # Add mock processed behavioral data
    speed_data = np.random.random(15)
    speed_time_series = pynwb.TimeSeries(
        name="speed",
        data=speed_data,
        rate=1.,
        description="The speed of the subject measured over time.",
        unit="m/s",
    )
    behavioral_time_series = pynwb.behavior.BehavioralTimeSeries(
        time_series=speed_time_series,
        name="BehavioralTimeSeries",
    )
    
    behavior_module.add(behavioral_time_series)

    # Use the export method to write the modified NWBFile object to a new file path. 
    # The original file is not modified
    # file_path_modified = file_path.split(".nwb")[0] + "_modified.nwb"
    file_path_modified = "modified_file.nwb"
    with pynwb.NWBHDF5IO(file_path_modified, mode="w") as export_io:
        export_io.export(src_io=read_io, nwbfile=nwbfile)

    print(f"Original file size: {get_file_size_in_mb(file_path)} MB")
    print(f"Modified file size: {get_file_size_in_mb(file_path_modified)} MB")

In [ ]:
# Read the modified NWB file and confirm the modifications
read_io = pynwb.NWBHDF5IO(file_path_modified, mode="r")
nwbfile = read_io.read()
nwbfile

In [ ]:
# Remember to close the file if you are exploring it outside of the context manager mode
read_io.close()

# Upload the modified NWB files to DANDI archive (staging)

Now we can upload the modified NWB files to DANDI archive. Let's upload them to the same dandiset you've been using, for simplicity.

In this session, you will use the [dandi cli](https://www.dandiarchive.org/handbook/12_download/#using-the-python-cli-client) to:
- Organize the files within the dandiset, which will automatically name the files copies using the DANDI convention.
- Validate the files
- Upload the files to DANDI archive. Don't forget to **set the right value for your DANDI_API_KEY** in the code below.

In [ ]:
bash_command = f'dandi organize -d {dandiset_id} -f move --required-field session_id .'

!{bash_command}

In [ ]:
bash_command = f'dandi validate {dandiset_id}'

!{bash_command}

In [ ]:
import os


# Save your API key for Dandi staging as an environment variable
os.environ["DANDI_API_KEY"] = "5e79c2cba4fd7ded10c1e6e3c577130669b98ee0"

# Run the bash command
bash_command = f'dandi upload -i dandi-staging {dandiset_id}'

!{bash_command}

# Explore the modified data files in DANDI archive and Neurosift

Once the modified data has been uploaded to DANDI archive, it becomes immediately available to be read and visualized online!

For that we will use [Neurosift](https://neurosift.app), a web app that allows you to explore and visualize all the data stored in the DANDI archive. Follow these steps:
- Run the cell below to get the url for your dandiset.
- Navigate to `Files` under `Dandiset Actions`
- Over the modified file, click on `Open with` then `Neurosift`

Alternatively, you can open [Neurosift](https://neurosift.app) and navigate through all available dandisets directly from there!

In [ ]:
import yaml

# Get the dandiset url from the YAML file
yaml_file_path = f"{dandiset_id}/dandiset.yaml"
with open(yaml_file_path, 'r') as file:
    yaml_dict = yaml.safe_load(file)
    print(yaml_dict["url"])